In [54]:
import torch
import torch.nn.functional as F

from Encoder import Encoder, MultiLayerEdgeGAT
from Env import BatchVRPEnvs
from Action import ActionSelector

from utils import load_model

from params import small_params, medium_params, large_params
from params import k_distance_nearest_neighbors_percent, k_time_nearest_neighbors_percent
from params import device, max_workers

In [2]:
ENV_PARAMS = small_params
ENV_PARAMS['index'] = 0
NUM_CUSTOMERS = ENV_PARAMS['num_customers']

In [3]:
PG_ENCODER_PATH = './train_model/models/check_point_encoder.pth'
PG_ACTION_PATH = './train_model/models/check_point_action_selector.pth'

# Environment

In [4]:
env = BatchVRPEnvs(
    max_workers=max_workers, 
    **ENV_PARAMS)
env.reset(generate=True)

In [26]:
def run_batch(env, encoder, action_selector, mode, generate, device):
    """
    执行一次完整的 sampling batch (即跑完一遍所有的 instances)
    """
    with torch.no_grad():
        env.reset(generate=generate)
        batch_size = env.batch_size

        env_encode(encoder, env.batch_customer_data, env.batch_company_data, env.wait_times)
        env.update_parameters(encoder.batch_distance_matrices, encoder.batch_num_nodes)
    
        # 记录 instance 是否结束
        instance_status = np.zeros(batch_size, dtype=bool)
        # 记录 reward
        reward_info = torch.zeros(batch_size, dtype=torch.float, device=device)
        # 记录 log_probs
        log_probs_info = torch.zeros(batch_size, dtype=torch.float, device=device)
        # 记录选择
        action_info = []
        # 记录时间步
        t = 0
        while not instance_status.all():
            current_batch_status = env.get_current_batch_status()
            current_vehicle_embeddings, current_customer_embeddings = encoder.get_current_batch_state(include_global=False, **current_batch_status)
            batch_neg_inf_mask = return_batch_neg_inf_masks(env).to(device)
            actions, log_probs = action_selector(
                current_vehicle_embeddings,
                current_customer_embeddings,
                batch_neg_inf_mask,
                mode=mode
            )
            if log_probs is not None:
                log_probs_info = log_probs_info + log_probs.sum(dim=1)
    
            # 执行选择的动作，并更新环境
            actions = actions.detach().cpu().numpy()
            action_info.append(actions)
            step_rewards, _, _, _ = batch_steps(env, actions, instance_status, device)
            reward_info = reward_info + step_rewards
    
            # print(f"{t} finished number: {instance_status.sum().item()}")
            t += 1

    return reward_info, log_probs_info, action_info

# Google OR Tool

In [33]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import numpy as np
from itertools import zip_longest

def construct_data_for_or_tool(origin_data, factor):
    customer_data = origin_data['customer_data']
    company_data = origin_data['company_data']
    
    data = {}
    data['locations'] = customer_data[['X','Y']].values.copy()
    data['demands'] = customer_data['Demand'].values.copy()
    data['demands'] = [int(x * factor) for x in data['demands']]
    data['start_times'] = customer_data['Start_Time_Window'].values.copy()
    data['start_times'] = [int(x * factor) for x in data['start_times']]
    data['end_times'] = customer_data['End_Time_Window'].values.copy()
    data['end_times'] = [int(x * factor) for x in data['end_times']]
    data['alpha'] = customer_data['Alpha'].values.copy()
    data['alpha'] = [int(x * factor) for x in data['alpha']]
    data['beta'] = customer_data['Beta'].values.copy()
    data['beta'] = [int(x * factor) for x in data['beta']]
    
    num_vehicles = company_data['Num_Vehicles']
    vehicle_capacities = company_data['Vehicle_Capacity']
    data['num_vehicles'] = int(num_vehicles)
    data['vehicle_capacities'] = [int(vehicle_capacities) * factor for _ in range(num_vehicles)]
    data['depot'] = 0
    return data


# Compute distance matrix (assuming 1:1 ratio between distance and time)
def compute_distance_matrix(locations, factor):
    distance_matrix = np.zeros((len(locations), len(locations)))
    for from_counter, from_node in enumerate(locations):
        for to_counter, to_node in enumerate(locations):
            if from_counter != to_counter:
                # Multiply the Euclidean distance by the factor
                distance_matrix[from_counter][to_counter] = int(np.linalg.norm(np.array(from_node) - np.array(to_node)) * factor)
            else:
                distance_matrix[from_counter][to_counter] = 0
    return distance_matrix.astype(int)


def print_solution(data, manager, routing, solution, factor, printout):
    """Prints solution on console, including on-time, early, and late arrival details and penalties."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    total_penalty = 0
    time_dimension = routing.GetDimensionOrDie("Time")
    actions_list = []

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        route_penalty = 0
        vehicle_actions = []

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index] / factor

            # Get cumulative time variable for the node
            arrival_time = solution.Value(time_dimension.CumulVar(index)) / factor
            time_window = (
                data['start_times'][node_index] / factor,
                data['end_times'][node_index] / factor,
                data['alpha'][node_index] / factor,
                data['beta'][node_index] / factor
            )
            lower_bound, upper_bound = time_window[0], time_window[1]
            lower_penalty, upper_penalty = time_window[2], time_window[3]

            # Determine if the arrival is early, on-time, or late
            if arrival_time < lower_bound:
                arrival_status = "Early"
                penalty = (lower_bound - arrival_time) * lower_penalty
            elif arrival_time > upper_bound:
                arrival_status = "Late"
                penalty = (arrival_time - upper_bound) * upper_penalty
            else:
                arrival_status = "On-time"
                penalty = 0

            route_penalty += penalty
            total_penalty += penalty

            # Format the output for this node
            if printout:
                plan_output += (
                    f"Node {node_index}:"
                    f"  Load: {round(route_load, 2)}"
                    f"  Arrival: {round(arrival_time, 2)}"
                    f"  Status: {arrival_status}"
                    f"  Penalty: {round(penalty, 2)}"
                    f"  Time Window: {(round(lower_bound, 2), round(upper_bound, 2))}"
                    f"  Time Window Penalty: {round(lower_penalty, 2), round(upper_penalty, 2)}\n"
                )

            # Move to the next index
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id) / factor
            vehicle_actions.append(index)

        # Last node in the route (depot)
        node_index = manager.IndexToNode(index)
        if printout:
            plan_output += f"Node {node_index}:"
            plan_output += f"  Load: {round(route_load, 2)}"
            plan_output += f"  Distance of the route: {round(route_distance, 2)}"
            plan_output += f"  Load of the route: {round(route_load, 2)}"
            plan_output += f"  Penalty of the route: {round(route_penalty, 2)}\n"
            print(plan_output)

        total_distance += route_distance
        total_load += route_load
        actions_list.append(vehicle_actions)
        
    print(f"Total distance of all routes: {round(total_distance, 4)}m")
    print(f"Total load of all routes: {round(total_load, 4)}")
    print(f"Total penalty of all routes: {round(total_penalty, 4)}")
    print(f"Total Cost of all routes: {round(total_distance + total_penalty, 4)}")

    actions_list = list(zip_longest(*actions_list, fillvalue=0))

    return total_distance, penalty, actions_list


def solve_vrp(data, factor=1000, printout=True):
    """Solve the CVRP problem."""
    data = construct_data_for_or_tool(data, factor=factor)
    data['distance_matrix'] = compute_distance_matrix(data['locations'], factor=factor)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        distance = data["distance_matrix"][from_node][to_node]
        return distance

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Add soft time windows constraint
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    time_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.AddDimension(
        time_callback_index,
        0,  # allow waiting time
        100 * factor,  # maximum travel time per vehicle
        False,  # Don't force start cumul to zero
        "Time",
    )

    time_dimension = routing.GetDimensionOrDie("Time")
    for location_idx in range(len(data['locations'])):
        if location_idx == data["depot"]:
            continue  # Skip the depot as it usually doesn't have time windows
        index = manager.NodeToIndex(location_idx)
        # Set hard lower bound (earliest start time) for the time window
        time_dimension.SetCumulVarSoftLowerBound(index, data['start_times'][location_idx], data['alpha'][location_idx])
        # Set soft upper bound (latest time) with penalty for late arrival
        time_dimension.SetCumulVarSoftUpperBound(index, data['end_times'][location_idx], data['beta'][location_idx])

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(10)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        total_distance, penalty, actions_list = print_solution(data, manager, routing, solution, factor, printout)
        return total_distance, penalty, actions_list
    else:
        print("No Solution Found!")

# Strategy Gradient

## Params

In [28]:
# --------- encoder -----------
# MultiLayerEdge
out_feats = 128
MultiLayerEdgeGATParams = {
    'in_feats': 11,
    'edge_feats': 10,
    'units': 128,
    'num_heads': 8,
    'num_layers': 2,
    'feat_drop': 0.0,
    'attn_drop': 0.0,
    'edge_drop': 0.0,
    'activation': F.leaky_relu
}
embedding_dim = out_feats
# --------- decoder -----------
# action
action_heads = 8
dynamic_vehicle_dim = 2
dynamic_customer_dim = 1

## models

In [29]:
k_neighbors_percent = 0.1

# encoder
pg_encoder = Encoder(
    encoder_model=MultiLayerEdgeGAT,
    encoder_params=MultiLayerEdgeGATParams,
    k_distance_nearest_neighbors_percent=k_distance_nearest_neighbors_percent,
    k_time_nearest_neighbors_percent=k_time_nearest_neighbors_percent,
    device=device
)
_ = pg_encoder.to(device)

# action selector
pg_action_selector = ActionSelector(
    embedding_dim=embedding_dim,
    heads=action_heads,
    dynamic_vehicle_dim=dynamic_vehicle_dim,
    dynamic_customer_dim=dynamic_customer_dim
)
_ = pg_action_selector.to(device)

pg_encoder = load_model(pg_encoder, PG_ENCODER_PATH)
pg_action_selector = load_model(pg_action_selector, PG_ACTION_PATH)

## Run

In [30]:
from train_model.train_utils import return_batch_neg_inf_masks, batch_steps, env_encode, replace_baseline_model

In [31]:
sampling_reward_info, _, sampling_action_info = run_batch(env, pg_encoder, pg_action_selector, mode='sampling', generate=False, device=device)
greedy_reward_info, _, greedy_action_info = run_batch(env, pg_encoder, pg_action_selector, mode='greedy', generate=False, device=device)

# Experiments

In [62]:
env_id = 150
vrp_env = env.envs[env_id]
vrp_customer_data = vrp_env.customer_data
vrp_company_data = vrp_env.company_data
vrp_travel_time_matrix = vrp_env.travel_time_matrix
vrp_customer_demand = vrp_customer_data['Demand'].values

## OR Tool

In [63]:
_, _, or_action = solve_vrp(
    data={'customer_data': vrp_customer_data, 'company_data': vrp_company_data}, 
    printout=False
)

Objective: 1656462
Total distance of all routes: 69.948m
Total load of all routes: 98.466
Total penalty of all routes: 1.5865
Total Cost of all routes: 71.5345


## Compare

In [34]:
ACTION_LIST = [action[env_id] for action in sampling_action_info]

In [35]:
total_distance = 0
penalty = 0
vehicle_id_list = list(range(vrp_company_data['Num_Vehicles']))
customers_id_list = list(range(1, vrp_company_data['Num_Customers']+1))
current_points = [0 for _ in vehicle_id_list]
current_time_elapsed = [0 for _ in vehicle_id_list]
current_capacities = [vrp_company_data['Vehicle_Capacity'] for _ in vehicle_id_list]
current_customer_demand = vrp_customer_demand.copy()

for idx, action in enumerate(ACTION_LIST):
    for vehicle_id in vehicle_id_list:
        curreent_location = current_points[vehicle_id]
        target_location = action[vehicle_id]
        distance = vrp_travel_time_matrix[curreent_location, target_location]
        if curreent_location == target_location:
            continue
        total_distance += distance
        current_time_elapsed[vehicle_id] += distance
        vehicle_time = current_time_elapsed[vehicle_id]
        if target_location in customers_id_list:
            time_windows = vrp_customer_data.iloc[target_location, 4:6]
            if vehicle_time < time_windows['Start_Time_Window']:
                penalty += (time_windows['Start_Time_Window'] - vehicle_time) * vrp_customer_data.iloc[target_location, 6]
            elif vehicle_time > time_windows['End_Time_Window']:
                penalty += (vehicle_time - time_windows['End_Time_Window']) * vrp_customer_data.iloc[target_location, 7]
            customer_demand = current_customer_demand[target_location]
            vehicle_capacity = current_capacities[vehicle_id]
            current_capacities[vehicle_id] = max(vehicle_capacity - customer_demand, 0)
            current_customer_demand[target_location] = max(customer_demand - vehicle_capacity, 0)
        current_points[vehicle_id] = target_location
print(f"Total Distance {total_distance}, Penalty {penalty}, Total Cost {total_distance + penalty}")

if sum(current_customer_demand) != 0:
    print("顾客需求没有被完全满足！")
    print("顾客需求：", current_customer_demand)
    print("剩余容量：", current_capacities)

Total Distance 162.83767658966985, Penalty 328.26656021341546, Total Cost 491.1042368030853
